In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
% matplotlib inline

## 读取数据，并示例
items = pd.read_csv(os.path.join(os.getcwd(), 'Data', 'tianchi_fresh_comp_train_item.csv'))
items.head(5)

,item_id,item_geohash,item_category
0,100002303,NaN,3368
1,100003592,NaN,7995
2,100006838,NaN,12630
3,100008089,NaN,7791
4,100012750,NaN,9614


In [8]:
users = pd.read_csv(os.path.join(os.getcwd(), 'Data', 'tianchi_fresh_comp_train_user.csv'), dtype={'user_id': object, 'item_id': object, 'behavior_type ': object})
users['date'] = users.time.map(lambda x:x.split(' ')[0])
users['hours'] = users.time.map(lambda x:x.split(' ')[-1])
users = users.drop(['time'], axis = 1)
users.head(5)

,user_id,item_id,behavior_type,user_geohash,item_category,date,hours
0,10001082,285259775,1,97lk14c,4076,2014-12-08,18
1,10001082,4368907,1,NaN,5503,2014-12-12,12
2,10001082,4368907,1,NaN,5503,2014-12-12,12
3,10001082,53616768,1,NaN,9762,2014-12-02,15
4,10001082,151466952,1,NaN,5232,2014-12-12,11


In [3]:
# 通过时间划分训练集和测试集
def SplitTrainandTestData(users, date):
    test_data = users[users['date'] == date]
    train_data = users[users['date'] < date]
    return(train_data, test_data)

train_data_12_18, test_data_12_18 = SplitTrainandTestData(users, '2014-12-18')
print(train_data_12_18.head(5))
test_data_12_18.head(5)

    user_id    item_id  behavior_type user_geohash  item_category  \
0  10001082  285259775              1      97lk14c           4076   
1  10001082    4368907              1          NaN           5503   
2  10001082    4368907              1          NaN           5503   
3  10001082   53616768              1          NaN           9762   
4  10001082  151466952              1          NaN           5232   

            time        date hours  
0  2014-12-08 18  2014-12-08    18  
1  2014-12-12 12  2014-12-12    12  
2  2014-12-12 12  2014-12-12    12  
3  2014-12-02 15  2014-12-02    15  
4  2014-12-12 11  2014-12-12    11  


,user_id,item_id,behavior_type,user_geohash,item_category,time,date,hours
217,100029775,247380548,1,9t4qcck,10223,2014-12-18 13,2014-12-18,13
232,100029775,247380548,2,9t4qc3g,10223,2014-12-18 13,2014-12-18,13
250,100029775,205264014,1,9t4qccb,1863,2014-12-18 13,2014-12-18,13
274,100029775,205264014,1,9t4qcc0,1863,2014-12-18 13,2014-12-18,13
317,100029775,87557153,1,9t4qcbm,10894,2014-12-18 13,2014-12-18,13


In [10]:
users = users.groupby(['user_id', 'item_id']).sum().reset_index()

In [11]:
users.head(5)

,user_id,item_id,behavior_type,item_category
0,100002985,101814844,2,6762
1,100002985,11906278,5,11394
2,100002985,143305462,2,6762
3,100002985,147599246,6,31293
4,100002985,157287038,2,7596
